In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt

In [3]:
%matplotlib widget

In [4]:
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None

# Checking for bugs

In [5]:
name_of_evil_file = "20180919_FC_LFP2_cen14_01_cc_01.res"
datapath = "/Users/jepe/scripting/cellpy/dev_data/bugfixing"

In [6]:
import os
import sys
import time
from pathlib import Path
print(f"running {sys.argv[0]}")

import cellpy
from cellpy import log
from cellpy import cellreader
from cellpy.parameters import prms

prms.Reader.use_cellpy_stat_file = False
#prms.Reader.cycle_mode = "cathode"
prms.Reader.sorted_data = False
log.setup_logging(default_level="INFO", custom_log_dir=os.getcwd())

filename = Path(datapath) / name_of_evil_file

assert os.path.isfile(filename)

running /Users/jepe/miniconda3/envs/cellpy/lib/python3.6/site-packages/ipykernel_launcher.py


In [7]:
d = cellreader.CellpyData()
d.from_raw(filename)
d.set_mass(0.12)
d.make_step_table()
d.make_summary()

# checking extracting cycles
n = d.get_number_of_cycles()
c = d.get_cycle_numbers()

(cellpy) - created CellpyData instance
(cellpy) - assuming cycling anode half-cell (discharge before charge)


/Users/jepe/scripting/cellpy/cellpy/readers/cellreader.py:2967: UserWarning: Cycle 11 is missing!
  warnings.warn(f"Cycle {j+1} is missing!")
/Users/jepe/scripting/cellpy/cellpy/readers/cellreader.py:2967: UserWarning: Cycle 22 is missing!
  warnings.warn(f"Cycle {j+1} is missing!")
/Users/jepe/scripting/cellpy/cellpy/readers/cellreader.py:2967: UserWarning: Cycle 33 is missing!
  warnings.warn(f"Cycle {j+1} is missing!")
/Users/jepe/scripting/cellpy/cellpy/readers/cellreader.py:2967: UserWarning: Cycle 44 is missing!
  warnings.warn(f"Cycle {j+1} is missing!")
/Users/jepe/scripting/cellpy/cellpy/readers/cellreader.py:2967: UserWarning: Cycle 55 is missing!
  warnings.warn(f"Cycle {j+1} is missing!")
/Users/jepe/scripting/cellpy/cellpy/readers/cellreader.py:2967: UserWarning: Cycle 66 is missing!
  warnings.warn(f"Cycle {j+1} is missing!")
/Users/jepe/scripting/cellpy/cellpy/readers/cellreader.py:2967: UserWarning: Cycle 77 is missing!
  warnings.warn(f"Cycle {j+1} is missing!")
/Users

In [8]:

current = d.dataset.dfdata
steps = d.dataset.step_table

In [27]:
fig, ax = plt.subplots()
for cycle in c:
    print(f"{cycle}", end=".")
    ch, volt_ch = d.get_ccap(cycle=cycle)
    dc, volt_dc = d.get_dcap(cycle=cycle)
    ax.plot(ch.values, volt_ch.values)
    ax.plot(dc.values, volt_dc.values)
   

FigureCanvasNbAgg()

1.2.3.4.5.6.7.8.9.10.12.13.14.15.16.17.18.19.20.21.23.24.25.26.27.28.29.30.31.32.34.35.36.37.38.39.40.41.42.43.45.46.47.48.49.50.51.52.53.54.56.57.58.59.60.61.62.63.64.65.67.68.69.70.71.72.73.74.75.76.78.79.80.81.82.83.84.85.86.87.89.90.91.92.93.94.95.96.97.98.

In [37]:
def extract_dqdv(cell_data, extract="charge", last_cycle=None):
    """Simple wrapper around the cellpy.utils.ica.dqdv function."""

    from cellpy.utils.ica import dqdv
    list_of_cycles = cell_data.get_cycle_numbers()
    if last_cycle is not None:
        list_of_cycles = [c for c in list_of_cycles if c <= int(last_cycle)]
        lprint(f"only processing up to cycle {last_cycle}")
        lprint(f"you have {len(list_of_cycles)} cycles to process")
    out_data = []
    for cycle in list_of_cycles:
        if extract=="charge":
            c, v = cell_data.get_ccap(cycle)
        else:
            c, v = cell_data.get_dcap(cycle)
        if v.any():
            try:
                v, dq = dqdv(v, c)
                v = v.tolist()
                dq = dq.tolist()
            except IndexError or OverflowError as e:
                v = list()
                dq = list()
                print(" -could not process this (cycle %i)" % cycle)
                print(" %s" % e)

            header_x = "dQ cycle_no %i" % cycle
            header_y = "voltage cycle_no %i" % cycle
            dq.insert(0, header_x)
            v.insert(0, header_y)

            out_data.append(v)
            out_data.append(dq)
        else:
            print(f"Empty step encountered for cycle={cycle}")
    return out_data

In [38]:
o1 = extract_dqdv(d, "discharge")
o2 = extract_dqdv(d, "charge")

/Users/jepe/miniconda3/envs/cellpy/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [66]:
import warnings
warnings.filterwarnings("error")

In [67]:

from cellpy.utils.ica import dqdv
c, v = d.get_ccap(2)
v2, dq = dqdv(v, c)

FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

In [57]:
fig2, (ax1, ax2) = plt.subplots(nrows=2, sharex=True)
ax1.plot(v2, dq)
ax2.plot(v, c)

/Users/jepe/miniconda3/envs/cellpy/lib/python3.6/site-packages/ipywidgets/widgets/widget.py:411: DeprecationWarning: Passing unrecoginized arguments to super(FigureCanvasNbAgg).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


FigureCanvasNbAgg()

In [60]:
dc, v3 = d.get_dcap(2)
v4, dq2 = dqdv(v3, dc)

/Users/jepe/miniconda3/envs/cellpy/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [61]:
fig3, (ax3, ax4) = plt.subplots(nrows=2, sharex=True)
ax3.plot(v4, dq2)
ax4.plot(v3, dc)

/Users/jepe/miniconda3/envs/cellpy/lib/python3.6/site-packages/ipywidgets/widgets/widget.py:411: DeprecationWarning: Passing unrecoginized arguments to super(FigureCanvasNbAgg).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


FigureCanvasNbAgg()

In [69]:
print(f"NOTE! get_ccap returns {type(v)} while dqdv returns {type(v2)}")
print("NOTE! also important to note that numpy issues a deprecation warning when running",
      "the scipy savgol_filter")

NOTE! get_ccap returns <class 'pandas.core.series.Series'> while dqdv returns <class 'numpy.ndarray'>
NOTE! also important to note that numpy issues a deprecation warning when running the scipy savgol_filter


# SUMMARY
he dqdv routine should be improved in the future
- if possible, it should return a dataframe
- if possible, it should be an option to return a dataframe in tidy format with all cycles
- if possible, it should be an option to return a xyyyyyy...y frame (interpolated)